In [ ]:
import requests, json

workflow = json.load(open("z_image_turbo_workflow.json", "r", encoding="utf-8"))

# LLM 只改这里
workflow["45"]["inputs"]["text"] = "a cyberpunk city at night, neon lights"

res = requests.post(
    "http://127.0.0.1:8000/prompt",
    json={"prompt": workflow}
)

print(res.json())



{'prompt_id': 'e3169a7b-bc22-40c3-945f-d363802945d0', 'number': 1, 'node_errors': {}}


In [11]:
print(res.json()["prompt_id"])


e3169a7b-bc22-40c3-945f-d363802945d0


In [ ]:
import time
image_info = ""
prompt_id = res.json()['prompt_id']
while True:
    r = requests.get(f"http://127.0.0.1:8000/history/{prompt_id}")
    data = r.json()

    if prompt_id in data:
        print("生成完成")
        print(data[prompt_id])
        image_info = data[prompt_id]
        break

    time.sleep(0.5)

生成完成
{'prompt': [1, 'e3169a7b-bc22-40c3-945f-d363802945d0', {'39': {'class_type': 'CLIPLoader', 'inputs': {'clip_name': 'qwen_3_4b.safetensors', 'type': 'lumina2', 'device': 'default'}}, '40': {'class_type': 'VAELoader', 'inputs': {'vae_name': 'ae.safetensors'}}, '46': {'class_type': 'UNETLoader', 'inputs': {'unet_name': 'z_image_turbo_bf16.safetensors', 'weight_dtype': 'default'}}, '48': {'class_type': 'LoraLoaderModelOnly', 'inputs': {'model': ['46', 0], 'lora_name': 'pixel_art_style_z_image_turbo.safetensors', 'strength_model': 1.0}}, '47': {'class_type': 'ModelSamplingAuraFlow', 'inputs': {'model': ['48', 0], 'shift': 3.0}}, '41': {'class_type': 'EmptySD3LatentImage', 'inputs': {'width': 1024, 'height': 1024, 'batch_size': 1}}, '45': {'class_type': 'CLIPTextEncode', 'inputs': {'clip': ['39', 0], 'text': 'a cyberpunk city at night, neon lights'}}, '42': {'class_type': 'ConditioningZeroOut', 'inputs': {'conditioning': ['45', 0]}}, '44': {'class_type': 'KSampler', 'inputs': {'model': 

In [21]:

for node_id, output in image_info["outputs"].items():
    if "images" in output:
        image = output["images"][0]
        filename = image["filename"]
        print(filename)


z-image_00043_.png


In [22]:
img = requests.get(
    "http://127.0.0.1:8000/view",
    params={
        "filename": filename,
        "type": "output"
    }
)

with open("result.png", "wb") as f:
    f.write(img.content)


1464241